## Set up the dependencies

In [1]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [2]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [3]:
# Analytics results
import emeval.metrics.baseline_segmentation as embs

In [4]:
# For plots
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
%matplotlib inline

import IPython.display as ipyd

In [5]:
# For maps
import folium
import branca.element as bre

In [6]:
# For easier debugging while working on modules
import importlib

In [7]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format
import arrow
import numpy as np

In [8]:
THIRTY_MINUTES = 30 * 60
TEN_MINUTES = 10 * 60

## The spec

The spec defines what experiments were done, and over which time ranges. Once the experiment is complete, most of the structure is read back from the data, but we use the spec to validate that it all worked correctly. The spec also contains the ground truth for the legs. Here, we read the spec for the trip to UC Berkeley.

In [9]:
DATASTORE_LOC = "bin/data"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sd_la = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la")
sd_sj = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "car_scooter_brex_san_jose")
sd_ucb = eisd.FileSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, "train_bus_ebike_mtv_ucb")

After iterating over 1 entries, entry found
Found spec = Round trip car and bike trip in the South Bay
Evaluation ran from 2019-07-20T00:00:00-07:00 -> 2020-04-29T17:00:00-07:00
After iterating over 1 entries, entry found
Found spec = Multi-modal car scooter BREX trip to San Jose
Evaluation ran from 2019-07-20T00:00:00-07:00 -> 2020-04-29T17:00:00-07:00
After iterating over 1 entries, entry found
Found spec = Multimodal multi-train, multi-bus, ebike trip to UC Berkeley
Evaluation ran from 2019-07-16T00:00:00-07:00 -> 2020-04-30T00:00:00-07:00


## The views

There are two main views for the data - the phone view and the evaluation view. 

### Phone view

In the phone view, the phone is primary, and then there is a tree that you can traverse to get the data that you want. Traversing that tree typically involves nested for loops; here's an example of loading the phone view and traversing it. You can replace the print statements with real code. When you are ready to check this in, please move the function to one of the python modules so that we can invoke it more generally

In [10]:
importlib.reload(eipv)

<module 'emeval.input.phone_view' from '/Users/kshankar/e-mission/mobilitynet-analysis-scripts/emeval/input/phone_view.py'>

In [11]:
pv_la = eipv.PhoneView(sd_la)

-------------------- About to read transitions from server --------------------
Reading data for android phones
Loading transitions for phone ucb-sdb-android-1
Loading transitions for phone ucb-sdb-android-2
Loading transitions for phone ucb-sdb-android-3
Loading transitions for phone ucb-sdb-android-4
Reading data for ios phones
Loading transitions for phone ucb-sdb-ios-1
Loading transitions for phone ucb-sdb-ios-2
Loading transitions for phone ucb-sdb-ios-3
Loading transitions for phone ucb-sdb-ios-4
-------------------- About to fill calibration ranges --------------------
Processing data for android phones
Processing transitions for phone ucb-sdb-android-1
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-2
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-3
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-4
Filtered 86 total -> 0 calibration

-------------------- About to fill in location information --------------------
Processing data for android phones
Processing data for ios phones
Processing data for android phones
Processing data for ios phones
-------------------- About to fill in motion activity information --------------------
Processing data for android phones
Processing data for ios phones
Processing data for android phones
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been proc

In [12]:
pv_sj = eipv.PhoneView(sd_sj)

-------------------- About to read transitions from server --------------------
Reading data for android phones
Loading transitions for phone ucb-sdb-android-1
Loading transitions for phone ucb-sdb-android-2
Loading transitions for phone ucb-sdb-android-3
Loading transitions for phone ucb-sdb-android-4
Reading data for ios phones
Loading transitions for phone ucb-sdb-ios-1
Loading transitions for phone ucb-sdb-ios-2
Loading transitions for phone ucb-sdb-ios-3
Loading transitions for phone ucb-sdb-ios-4
-------------------- About to fill calibration ranges --------------------
Processing data for android phones
Processing transitions for phone ucb-sdb-android-1
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-2
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-3
Filtered 86 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-4
Filtered 86 total -> 0 calibration

('bus trip with e-scooter access_0', 4292.051233053207, <Arrow [2019-08-07T13:40:16.767767-07:00]>, <Arrow [2019-08-07T14:51:48.819000-07:00]>)
('START_EVALUATION_TRIP', 'freeway_driving_weekday', 1583349616.581, 1583349616.665268)
('STOP_EVALUATION_TRIP', 'freeway_driving_weekday', 1583351508.552, 1583351508.554702)
('START_EVALUATION_TRIP', 'bus trip with e-scooter access', 1583364868.374, 1583364868.447122)
('STOP_EVALUATION_TRIP', 'bus trip with e-scooter access', 1583370583.544, 1583370583.5469341)
All ranges are complete, nothing to change
{'bus trip with e-scooter access', 'freeway_driving_weekday'}
ios: Found 2 trips for evaluation fixed:ACCURACY_CONTROL_6
('freeway_driving_weekday_0', 1891.8894340991974, <Arrow [2020-03-04T11:20:16.665268-08:00]>, <Arrow [2020-03-04T11:51:48.554702-08:00]>)
('bus trip with e-scooter access_0', 5715.099812030792, <Arrow [2020-03-04T15:34:28.447122-08:00]>, <Arrow [2020-03-04T17:09:43.546934-08:00]>)
-------------------- About to fill in evaluat

-------------------- About to fill in location information --------------------
Processing data for android phones
Processing data for ios phones
Processing data for android phones
Processing data for ios phones
-------------------- About to fill in motion activity information --------------------
Processing data for android phones
Processing data for ios phones
Processing data for android phones
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been proc

In [13]:
pv_ucb = eipv.PhoneView(sd_ucb)

-------------------- About to read transitions from server --------------------
Reading data for android phones
Loading transitions for phone ucb-sdb-android-1
Loading transitions for phone ucb-sdb-android-2
Loading transitions for phone ucb-sdb-android-3
Loading transitions for phone ucb-sdb-android-4
Reading data for ios phones
Loading transitions for phone ucb-sdb-ios-1
Loading transitions for phone ucb-sdb-ios-2
Loading transitions for phone ucb-sdb-ios-3
Loading transitions for phone ucb-sdb-ios-4
-------------------- About to fill calibration ranges --------------------
Processing data for android phones
Processing transitions for phone ucb-sdb-android-1
Filtered 102 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-2
Filtered 90 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-3
Filtered 90 total -> 0 calibration transitions 
Processing transitions for phone ucb-sdb-android-4
Filtered 90 total -> 0 calibratio

Processing data for ios phones
-------------------- About to fill in location information --------------------
Processing data for android phones
Processing data for ios phones
Processing data for android phones
Processing data for ios phones
-------------------- About to fill in motion activity information --------------------
Processing data for android phones
Processing data for ios phones
Processing data for android phones
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
motion activity has not been processed, copying write_ts -> ts
mo

Filling label ucb-sdb-android-2 for OS android
Filling label ucb-sdb-android-3 for OS android
Filling label ucb-sdb-android-4 for OS android
Filling label ucb-sdb-ios-1 for OS ios
Filling label ucb-sdb-ios-2 for OS ios
Filling label ucb-sdb-ios-3 for OS ios
Filling label ucb-sdb-ios-4 for OS ios
-------------------- Done populating information from server --------------------


In [14]:
import emeval.analysed.phone_view as eapv

In [15]:
importlib.reload(eapv)

<module 'emeval.analysed.phone_view' from '/Users/kshankar/e-mission/mobilitynet-analysis-scripts/emeval/analysed/phone_view.py'>

In [16]:
MASTER_DATA_LOC = "bin/data/master_9b70c97"
eisd.FileSpecDetails(MASTER_DATA_LOC, AUTHOR_EMAIL)

In [17]:
av_la = eapv.create_analysed_view(pv_la, eisd.FileSpecDetails("bin/data/master_9b70c97", AUTHOR_EMAIL), "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")

Finished copying unimodal_trip_car_bike_mtv_la, starting overwrite
Found spec = Round trip car and bike trip in the South Bay
Evaluation ran from 2019-07-20T00:00:00-07:00 -> 2020-04-29T17:00:00-07:00
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
android dict_keys(['ucb-sdb-android-1', 'ucb-sdb-android-2', 'ucb-sdb-android-3', 'ucb-sdb-android-4'])
     -*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
     ucb-sdb-android-1 accuracy_control dict_keys(['role', 'manual/evaluation_transition', 'calibration_transitions', 'calibration_ranges', 'evaluation_transitions', 'evaluation_ranges'])
         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transition_df'])
         fixed:ACCURACY

Before filtering, trips = [('2019-07-28T10:21:47.338000-07:00', '2019-07-28T10:32:22-07:00'), ('2019-07-28T11:50:20.124734-07:00', '2019-07-28T12:01:27-07:00'), ('2019-07-28T12:01:57-07:00', '2019-07-28T12:01:57-07:00'), ('2019-07-28T12:02:28-07:00', '2019-07-28T12:03:58-07:00'), ('2019-07-28T12:05:28-07:00', '2019-07-28T12:10:00-07:00')]
Filter range = 2019-07-28T11:48:06.675345-07:00 -> 2019-07-28T12:09:44.829831-07:00
After filtering, trips = ['2019-07-28T11:50:20.124734-07:00', '2019-07-28T12:01:57-07:00', '2019-07-28T12:02:28-07:00', '2019-07-28T12:05:28-07:00']
         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transition_df'])
         

Before filtering, trips = [('2019-08-11T17:53:59.032591-07:00', '2019-08-11T18:06:13.990799-07:00'), ('2019-08-11T18:06:14.990764-07:00', '2019-08-11T18:07:53.987217-07:00'), ('2019-08-11T19:12:38.044344-07:00', '2019-08-11T19:15:15.997580-07:00'), ('2019-08-11T19:15:16.997539-07:00', '2019-08-11T19:15:20.997372-07:00'), ('2019-08-11T19:15:21.997330-07:00', '2019-08-11T19:21:48.998054-07:00'), ('2019-08-11T19:21:49.998110-07:00', '2019-08-11T19:22:08.998388-07:00'), ('2019-08-11T19:22:09.998376-07:00', '2019-08-11T19:24:17.993807-07:00'), ('2019-08-11T19:24:18.993765-07:00', '2019-08-11T19:27:35.986123-07:00'), ('2019-08-11T19:27:36.986084-07:00', '2019-08-11T19:30:19.996613-07:00'), ('2019-08-11T19:30:22.996500-07:00', '2019-08-11T19:32:37.991355-07:00')]
Filter range = 2019-08-11T17:51:19.220633-07:00 -> 2019-08-11T18:07:09.679044-07:00
After filtering, trips = ['2019-08-11T17:53:59.032591-07:00', '2019-08-11T18:06:14.990764-07:00']
Before filtering, trips = [('2019-08-11T17:53:59.03

In [18]:
range_0 = av_la.map()["ios"]["ucb-sdb-ios-3"]["evaluation_ranges"][0]
for t in range_0["sensed_trip_ranges"]:
    print(arrow.get(t["start_ts"]).to("America/Los_angeles"), arrow.get(t["end_ts"]).to("America/Los_angeles"))
print("=======")
for s in range_0["sensed_section_ranges"]:
    print(arrow.get(s["start_ts"]).to("America/Los_angeles"), arrow.get(s["end_ts"]).to("America/Los_angeles"))
print("=======")

for t in range_0["evaluation_trip_ranges"]:
    print(arrow.get(t["start_ts"]).to("America/Los_angeles"), arrow.get(t["end_ts"]).to("America/Los_angeles"))
    for s in t["sensed_section_ranges"]:
        print("-------", arrow.get(s["start_ts"]).to("America/Los_angeles"), arrow.get(s["end_ts"]).to("America/Los_angeles"))

2019-07-27T19:01:45.481526-07:00 2019-07-27T19:21:37.427862-07:00
2019-07-27T19:01:45.481526-07:00 2019-07-27T19:21:37.427862-07:00
2019-07-27T17:38:54.143985-07:00 2019-07-27T17:54:56.504297-07:00
2019-07-27T18:59:17.435039-07:00 2019-07-27T19:20:57.464819-07:00
------- 2019-07-27T19:01:45.481526-07:00 2019-07-27T19:21:37.427862-07:00


In [19]:
# av_la = eapv.create_analysed_view(pv_la, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
av_sj = eapv.create_analysed_view(pv_sj, eisd.FileSpecDetails("bin/data/master_9b70c97", AUTHOR_EMAIL), "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
av_ucb = eapv.create_analysed_view(pv_ucb, eisd.FileSpecDetails("bin/data/master_9b70c97", AUTHOR_EMAIL), "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")
# av_ucb_reroute = eapv.create_analysed_view(pv_ucb_reroute, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/inferred_section")

Finished copying car_scooter_brex_san_jose, starting overwrite
Found spec = Multi-modal car scooter BREX trip to San Jose
Evaluation ran from 2019-07-20T00:00:00-07:00 -> 2020-04-29T17:00:00-07:00
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
android dict_keys(['ucb-sdb-android-1', 'ucb-sdb-android-2', 'ucb-sdb-android-3', 'ucb-sdb-android-4'])
     -*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
     ucb-sdb-android-1 accuracy_control dict_keys(['role', 'manual/evaluation_transition', 'calibration_transitions', 'calibration_ranges', 'evaluation_transitions', 'evaluation_ranges'])
         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transition_df'])
         fixed:ACCURACY_CON

Before filtering, trips = [('2019-08-07T09:21:47.112000-07:00', '2019-08-07T09:24:05-07:00'), ('2019-08-07T09:24:06-07:00', '2019-08-07T09:25:36-07:00'), ('2019-08-07T09:25:37-07:00', '2019-08-07T09:26:05-07:00'), ('2019-08-07T09:26:06-07:00', '2019-08-07T09:28:28-07:00'), ('2019-08-07T09:28:29-07:00', '2019-08-07T09:29:18-07:00'), ('2019-08-07T09:29:19-07:00', '2019-08-07T09:29:43-07:00'), ('2019-08-07T09:29:44-07:00', '2019-08-07T09:47:18-07:00'), ('2019-08-07T09:47:19-07:00', '2019-08-07T09:53:19.901000-07:00'), ('2019-08-07T11:19:17.788957-07:00', '2019-08-07T11:24:27-07:00'), ('2019-08-07T13:28:32.366548-07:00', '2019-08-07T13:40:21-07:00'), ('2019-08-07T13:40:52-07:00', '2019-08-07T13:41:30-07:00'), ('2019-08-07T13:41:31-07:00', '2019-08-07T13:46:12-07:00'), ('2019-08-07T13:46:13-07:00', '2019-08-07T13:46:24-07:00'), ('2019-08-07T13:46:25-07:00', '2019-08-07T13:48:21-07:00'), ('2019-08-07T13:49:47-07:00', '2019-08-07T13:50:12-07:00'), ('2019-08-07T13:50:14-07:00', '2019-08-07T13:

Before filtering, trips = [('2019-08-07T09:21:38.051000-07:00', '2019-08-07T09:26:01-07:00'), ('2019-08-07T09:26:02-07:00', '2019-08-07T09:28:11-07:00'), ('2019-08-07T09:28:12-07:00', '2019-08-07T09:47:19-07:00'), ('2019-08-07T09:47:20-07:00', '2019-08-07T09:48:20.642000-07:00'), ('2019-08-07T13:40:28.027609-07:00', '2019-08-07T13:43:30-07:00'), ('2019-08-07T13:43:31-07:00', '2019-08-07T13:44:15.469000-07:00'), ('2019-08-07T13:45:13-07:00', '2019-08-07T13:47:39-07:00'), ('2019-08-07T13:47:40-07:00', '2019-08-07T13:48:17-07:00'), ('2019-08-07T13:48:18-07:00', '2019-08-07T13:55:09-07:00'), ('2019-08-07T13:55:11-07:00', '2019-08-07T13:55:44-07:00'), ('2019-08-07T13:55:45-07:00', '2019-08-07T14:02:01-07:00'), ('2019-08-07T14:02:02-07:00', '2019-08-07T14:06:47.031000-07:00'), ('2019-08-07T14:07:04.400000-07:00', '2019-08-07T14:08:34-07:00'), ('2019-08-07T14:08:35-07:00', '2019-08-07T14:41:21-07:00'), ('2019-08-07T14:41:23-07:00', '2019-08-07T14:41:24-07:00'), ('2019-08-07T14:41:45-07:00', '

Before filtering, trips = [('2019-08-05T10:30:22.521594-07:00', '2019-08-05T10:32:50.994496-07:00'), ('2019-08-05T10:33:08.466827-07:00', '2019-08-05T10:34:27.990568-07:00'), ('2019-08-05T10:34:30.990447-07:00', '2019-08-05T10:34:48.989717-07:00'), ('2019-08-05T10:34:49.989676-07:00', '2019-08-05T10:55:58.992105-07:00'), ('2019-08-05T10:55:59.992069-07:00', '2019-08-05T10:59:18.455394-07:00'), ('2019-08-05T15:04:33.485448-07:00', '2019-08-05T15:13:55.998247-07:00'), ('2019-08-05T15:13:56.998283-07:00', '2019-08-05T15:14:05.998424-07:00'), ('2019-08-05T15:14:06.998425-07:00', '2019-08-05T15:22:49.995738-07:00'), ('2019-08-05T15:22:50.995700-07:00', '2019-08-05T15:22:54.995555-07:00'), ('2019-08-05T15:22:55.995518-07:00', '2019-08-05T15:26:44.987143-07:00'), ('2019-08-05T15:26:46.987070-07:00', '2019-08-05T15:34:52.985577-07:00'), ('2019-08-05T15:34:53.985543-07:00', '2019-08-05T16:09:15.989569-07:00'), ('2019-08-05T16:09:17.989504-07:00', '2019-08-05T16:21:31.436115-07:00')]
Filter rang

Before filtering, trips = [('2019-08-05T10:32:22.335140-07:00', '2019-08-05T10:55:34.997035-07:00'), ('2019-08-05T10:56:02.996000-07:00', '2019-08-05T10:57:49.991994-07:00'), ('2019-08-05T15:04:31.337241-07:00', '2019-08-05T15:09:56.988751-07:00'), ('2019-08-05T15:10:21.987585-07:00', '2019-08-05T15:10:21.987585-07:00'), ('2019-08-05T15:10:52.986137-07:00', '2019-08-05T15:10:52.986137-07:00'), ('2019-08-05T15:11:00.985764-07:00', '2019-08-05T15:13:08.979793-07:00'), ('2019-08-05T15:13:59.977416-07:00', '2019-08-05T15:13:59.977416-07:00'), ('2019-08-05T15:14:07.989907-07:00', '2019-08-05T15:22:42.979539-07:00'), ('2019-08-05T15:22:50.979212-07:00', '2019-08-05T15:22:50.979212-07:00'), ('2019-08-05T15:22:56.978963-07:00', '2019-08-05T15:26:47.992232-07:00'), ('2019-08-05T15:34:35.991219-07:00', '2019-08-05T15:34:42.990969-07:00'), ('2019-08-05T15:34:45.990864-07:00', '2019-08-05T16:08:43.994686-07:00'), ('2019-08-05T16:09:33.992770-07:00', '2019-08-05T16:18:29.994664-07:00')]
Filter rang

...

         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transition_df'])
         MAMFDC v/s HAMFDC:MAMFDC_2 MAMFDC v/s HAMFDC MAMFDC_2 3
Before filtering, trips = [('2020-02-06T08:23:15.894629-08:00', '2020-02-06T08:25:02.736740-08:00'), ('2020-02-06T08:26:00.821645-08:00', '2020-02-06T08:34:58.642714-08:00'), ('2020-02-06T08:35:36.623463-08:00', '2020-02-06T08:57:36.177141-08:00'), ('2020-02-06T18:42:26.341399-08:00', '2020-02-06T18:56:51.030249-08:00'), ('2020-02-06T18:57:10.358475-08:00', '2020-02-06T19:19:47.315187-08:00')]
Filter range = 2020-02-06T08:14:46.468857-08:00 -> 2020-02-06T10:30:08.723332-08:00
After filtering, trips = ['2020-02-06

         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transition_df'])
         HAHFDC v/s MAHFDC:MAHFDC_2 HAHFDC v/s MAHFDC MAHFDC_2 3
Before filtering, trips = [('2019-07-26T08:22:52.964088-07:00', '2019-07-26T08:22:52.964088-07:00'), ('2019-07-26T08:27:29.341601-07:00', '2019-07-26T08:39:20.898491-07:00'), ('2019-07-26T08:39:57.669632-07:00', '2019-07-26T08:46:06.468568-07:00'), ('2019-07-26T08:46:28.099438-07:00', '2019-07-26T08:51:20.000168-07:00'), ('2019-07-26T08:51:26.413524-07:00', '2019-07-26T09:11:37.049495-07:00'), ('2019-07-26T09:12:29.399212-07:00', '2019-07-26T09:19:59.901535-07:00'), ('2019-07-26T09:29:08.803343-07:00', '2019-07-2

Before filtering, trips = [('2019-09-17T08:13:51.789537-07:00', '2019-09-17T08:22:36.999940-07:00'), ('2019-09-17T08:37:11.363045-07:00', '2019-09-17T09:14:06.996105-07:00'), ('2019-09-17T09:14:39.994755-07:00', '2019-09-17T09:19:33.994082-07:00'), ('2019-09-17T09:20:26.998075-07:00', '2019-09-17T10:11:10.109991-07:00'), ('2019-09-17T10:14:47.395200-07:00', '2019-09-17T10:18:46.997222-07:00'), ('2019-09-17T10:35:34.181786-07:00', '2019-09-17T10:37:45.999127-07:00'), ('2019-09-17T10:39:30.994917-07:00', '2019-09-17T10:39:30.994917-07:00'), ('2019-09-17T13:49:09.097899-07:00', '2019-09-17T13:59:56.013239-07:00'), ('2019-09-17T16:32:23.005124-07:00', '2019-09-17T16:35:45.991191-07:00'), ('2019-09-17T16:37:47.985729-07:00', '2019-09-17T16:38:20.984250-07:00'), ('2019-09-17T16:55:09.005501-07:00', '2019-09-17T17:25:30.996922-07:00'), ('2019-09-17T17:26:48.993547-07:00', '2019-09-17T17:45:54.989123-07:00'), ('2019-09-17T17:46:12.988362-07:00', '2019-09-17T17:53:53.992627-07:00'), ('2019-09-1

Before filtering, trips = [('2019-12-03T08:19:06.616476-08:00', '2019-12-03T08:35:35.995366-08:00'), ('2019-12-03T08:35:43.731022-08:00', '2019-12-03T09:09:59.149219-08:00'), ('2019-12-03T09:12:04.560350-08:00', '2019-12-03T09:18:53.639832-08:00'), ('2019-12-03T09:25:08.570770-08:00', '2019-12-03T10:12:10.203423-08:00'), ('2019-12-03T10:15:11.470732-08:00', '2019-12-03T10:32:03.418188-08:00'), ('2019-12-03T10:32:09.901798-08:00', '2019-12-03T10:32:29.410172-08:00'), ('2019-12-03T10:32:35.895938-08:00', '2019-12-03T10:35:25.508647-08:00'), ('2019-12-03T14:15:19.779091-08:00', '2019-12-03T14:28:27.915522-08:00'), ('2019-12-03T16:18:29.018600-08:00', '2019-12-03T16:45:38.443121-08:00'), ('2019-12-03T17:18:39.599550-08:00', '2019-12-03T17:54:07.610096-08:00'), ('2019-12-03T17:54:15.528126-08:00', '2019-12-03T18:19:26.156419-08:00'), ('2019-12-03T18:26:43.493859-08:00', '2019-12-03T19:16:32.592792-08:00'), ('2019-12-03T19:16:39.033753-08:00', '2019-12-03T19:33:59.407157-08:00')]
Filter rang

Before filtering, trips = []
Filter range = 2019-07-24T07:51:42.185629-07:00 -> 2019-07-24T10:26:37.702858-07:00
After filtering, trips = []
Before filtering, trips = []
Filter range = 2019-07-24T14:12:16.706653-07:00 -> 2019-07-24T14:25:34.793104-07:00
After filtering, trips = []
Before filtering, trips = []
Filter range = 2019-07-24T16:37:07.662981-07:00 -> 2019-07-24T19:59:19.661789-07:00
After filtering, trips = []
         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transition_df'])
         fixed:POWER_CONTROL_1 HAHFDC v/s MAHFDC power_control_1 3
Before filtering, trips = []
Filter range = 2019-07-25T08:06:28.267119-07:00 -> 2019-07-25T10

In [20]:
def verify_range(eval_range):
    if len(eval_range["evaluation_trip_ranges"]) > len(eval_range["sensed_trip_ranges"]):
        print(f"MISSING TRIP: found {len(eval_range['sensed_trip_ranges'])}, expected {len(eval_range['evaluation_trip_ranges'])}")
    if len(eval_range["evaluation_trip_ranges"]) < len(eval_range["sensed_trip_ranges"]):
        print(f"EXTRA TRIP: found {len(eval_range['sensed_trip_ranges'])}, expected {len(eval_range['evaluation_trip_ranges'])}")
    for i, t in enumerate(eval_range["evaluation_trip_ranges"]):
        if len(t["evaluation_section_ranges"]) > len(t["sensed_section_ranges"]):
            print(f"MISSING SECTION: In trip {i}, found {len(t['sensed_section_ranges'])}, expected {len(t['evaluation_section_ranges'])}")
        if len(t["evaluation_section_ranges"]) < len(t["sensed_section_ranges"]):
            print(f"EXTRA SECTION: In trip {i}, found {len(t['sensed_section_ranges'])}, expected {len(t['evaluation_section_ranges'])}")

In [21]:
def verify_analysed_view(av):
    for phone_os, phone_map in av.map().items():
        print(15 * "=*")
        for phone_label, phone_detail_map in phone_map.items():
            for r in phone_detail_map["evaluation_ranges"]:
                print(8 * ' ', 30 * "=")
                print(8 * ' ',phone_os, phone_label, r["trip_id"], r["eval_common_trip_id"], r["eval_role"])
                if "control" not in r["eval_role"]:
                    verify_range(r)

In [22]:
verify_analysed_view(av_la)

=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_0 HAHFDC v/s HAMFDC accuracy_control_0
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_1 HAHFDC v/s HAMFDC accuracy_control_1
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_2 HAHFDC v/s HAMFDC accuracy_control_2
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_3 HAHFDC v/s MAHFDC accuracy_control_0
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_4 HAHFDC v/s MAHFDC accuracy_control_1
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_5 HAHFDC v/s MAHFDC accuracy_control_2
         android ucb-sdb-android-2 HAHFDC v/s HAMFDC:HAHFDC_0 HAHFDC v/s HAMFDC HAHFDC_0
MISSING SECTION: In trip 0, found 2, expected 3
EXTRA SECTION: In trip 1, found 7, expected 3
         android ucb-sdb-android-2 HAHFDC v/s HAMFDC:HAHFDC_1 HAHFDC v/s HAMFDC HAHFDC_1
MISSING SECTION: In trip 0, found 2, expected 3
MISSING SECTION: In trip 1, found 1, expected 3
         android uc

## There is only one missing trip; let's plot it to be sure

In [23]:
first_repetition = pv_la.map()["ios"]["ucb-sdb-ios-3"]["evaluation_ranges"][0]; first_repetition["trip_id"]

'HAHFDC v/s MAHFDC:MAHFDC_0'

In [24]:
missing_trip = first_repetition["evaluation_trip_ranges"][0]; print(missing_trip["trip_id"])

suburb_city_driving_weekend_0


In [25]:
missing_trip.keys()

dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'evaluation_section_ranges', 'battery_df', 'location_df', 'filtered_location_df', 'motion_activity_df', 'transition_df'])

In [26]:
missing_trip["location_df"]

,filter,floor,longitude,sensed_speed,ts,bearing,latitude,vaccuracy,fmt_time,accuracy,altitude,write_ts,hr


In [27]:
missing_trip["battery_df"]

,battery_status,ts,write_ts,battery_level_pct,hr
4,1,1564274608.430625,1564274608.430826,100.000000,0.095338
5,1,1564274608.499836,1564274608.500046,100.000000,0.095358
6,1,1564274608.812359,1564274608.812797,100.000000,0.095444
7,1,1564274608.954107,1564274608.954581,100.000000,0.095484
8,1,1564274609.243241,1564274609.243699,100.000000,0.095564
9,1,1564274609.411828,1564274609.412329,100.000000,0.095611
10,1,1564274609.500512,1564274609.501017,100.000000,0.095636


In [28]:
missing_trip["motion_activity_df"].ts.apply(lambda ts: arrow.get(ts).to("America/Los_angeles"))

1     2019-07-27T17:39:07.477725-07:00
2     2019-07-27T17:40:37.477725-07:00
3     2019-07-27T17:42:22.477725-07:00
4     2019-07-27T17:42:32.477724-07:00
5     2019-07-27T17:43:27.709919-07:00
6     2019-07-27T17:43:35.345936-07:00
7     2019-07-27T17:44:08.697577-07:00
8     2019-07-27T17:44:10.937719-07:00
9     2019-07-27T17:44:13.479989-07:00
10    2019-07-27T17:44:21.008866-07:00
11    2019-07-27T17:46:36.008866-07:00
12    2019-07-27T17:47:11.008867-07:00
13    2019-07-27T17:52:36.008867-07:00
14    2019-07-27T17:52:41.008867-07:00
15    2019-07-27T17:53:26.008867-07:00
Name: ts, dtype: object

## Checking the other specs

- Both of them have extra trips, no missing trips
- They have both missing and extra sections, but nothing where the sensed sections are zero

In [29]:
verify_analysed_view(av_sj)

=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_0 HAHFDC v/s HAMFDC accuracy_control_0
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_1 HAHFDC v/s HAMFDC accuracy_control_1
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_2 HAHFDC v/s HAMFDC accuracy_control_2
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_3 HAHFDC v/s MAHFDC accuracy_control_0
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_4 HAHFDC v/s MAHFDC accuracy_control_1
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_5 HAHFDC v/s MAHFDC accuracy_control_2
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_6 MAMFDC v/s HAMFDC accuracy_control_0
         android ucb-sdb-android-2 HAHFDC v/s HAMFDC:HAHFDC_0 HAHFDC v/s HAMFDC HAHFDC_0
EXTRA TRIP: found 3, expected 2
MISSING SECTION: In trip 0, found 2, expected 3
EXTRA SECTION: In trip 1, found 19, expected 6
         android ucb-sdb-android-2 HAHFDC v/s HAMFDC:HAHFDC_1 HAHFDC v/s H

In [30]:
verify_analysed_view(av_ucb)

=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_0 HAHFDC v/s HAMFDC accuracy_control_0
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_1 HAHFDC v/s HAMFDC accuracy_control_1
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_2 HAHFDC v/s HAMFDC accuracy_control_2
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_3 HAHFDC v/s MAHFDC accuracy_control_0
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_4 HAHFDC v/s MAHFDC accuracy_control_1
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_5 HAHFDC v/s MAHFDC accuracy_control_2
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_6 MAMFDC v/s HAMFDC accuracy_control_0
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_7 MAMFDC v/s HAMFDC accuracy_control_1
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_8 MAMFDC v/s HAMFDC accuracy_control_2
         android ucb-sdb-android-1 fixed:ACCURACY_CONTROL_9 MAMFDC v/s MAHFDC accuracy_control_0

         ios ucb-sdb-ios-1 fixed:ACCURACY_CONTROL_6 MAMFDC v/s MAHFDC accuracy_control_0
         ios ucb-sdb-ios-1 fixed:ACCURACY_CONTROL_7 MAMFDC v/s MAHFDC accuracy_control_1
         ios ucb-sdb-ios-1 fixed:ACCURACY_CONTROL_8 MAMFDC v/s MAHFDC accuracy_control_2
         ios ucb-sdb-ios-1 fixed:ACCURACY_CONTROL_9 MAMFDC v/s HAMFDC accuracy_control_0
         ios ucb-sdb-ios-1 fixed:ACCURACY_CONTROL_10 MAMFDC v/s HAMFDC accuracy_control_1
         ios ucb-sdb-ios-1 fixed:ACCURACY_CONTROL_11 MAMFDC v/s HAMFDC accuracy_control_2
         ios ucb-sdb-ios-2 HAHFDC v/s MAHFDC:HAHFDC_0 HAHFDC v/s MAHFDC HAHFDC_0
EXTRA TRIP: found 4, expected 3
MISSING SECTION: In trip 0, found 7, expected 10
EXTRA SECTION: In trip 2, found 16, expected 12
         ios ucb-sdb-ios-2 HAHFDC v/s MAHFDC:HAHFDC_1 HAHFDC v/s MAHFDC HAHFDC_1
EXTRA TRIP: found 4, expected 3
EXTRA SECTION: In trip 0, found 11, expected 10
EXTRA SECTION: In trip 2, found 19, expected 12
         ios ucb-sdb-ios-2 HAHFDC v/s MAHFDC: